In [6]:
import sqlite3
import re

def limpar_texto(texto):
    """
    Limpa o texto para melhorar o treinamento.
    Remove quebras de linha, espaços extras e caracteres especiais.
    """
    # Remove quebras de linha e tabs
    texto = texto.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    # Remove múltiplos espaços
    texto = re.sub(r'\s+', ' ', texto)
    # Remove caracteres especiais indesejados, mantendo pontuação básica
    texto = re.sub(r'[^a-zA-ZÀ-ÿ0-9,.!?;:()\'" ]', '', texto)
    # Remove espaços no início e fim
    texto = texto.strip()
    return texto

In [11]:

def limpar_e_atualizar_banco(db_path="projeto_fernando_pessoa/textos.db"):
    """
    Limpa todos os textos no banco e atualiza os registros.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Seleciona todos os textos com seus IDs
    cursor.execute("SELECT id, texto FROM textos")
    registros = cursor.fetchall()

    for registro_id, texto in registros:
        texto_limpo = limpar_texto(texto)
        # Atualiza o registro no banco
        cursor.execute("UPDATE textos SET texto = ? WHERE id = ?", (texto_limpo, registro_id))

    conn.commit()
    conn.close()
    print(f"{len(registros)} textos foram limpos e atualizados no banco.")

In [10]:

# Exemplo de uso
limpar_e_atualizar_banco()


1180 textos foram limpos e atualizados no banco.
